In [1]:
import torch
import numpy as np

In [2]:
num_inputs = 2
num_examples = 1000
true_w = torch.tensor([2, -3.4]).view(2,1)
true_b = torch.tensor(4.2)
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = torch.mm(features, true_w) + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
import torch.utils.data as Data

batch_size = 10

dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [4]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.8870, -2.0432],
        [ 2.1428,  0.4537],
        [ 0.2021, -2.0300],
        [-0.1116,  0.2102],
        [-1.4682, -0.8169],
        [-0.1325, -1.1115],
        [-0.3572,  0.7903],
        [ 1.6930, -0.4632],
        [ 0.2782, -0.2765],
        [ 0.2184,  0.6515]]) tensor([[ 9.3674],
        [ 6.9479],
        [11.5357],
        [ 3.2478],
        [ 4.0107],
        [ 7.7065],
        [ 0.7934],
        [ 9.1628],
        [ 5.6901],
        [ 2.4212]])


In [5]:
import torch.nn as nn

In [7]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    
    def forward(self, x):
        y = self.linear(x)
        return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.1766, -0.3211]], requires_grad=True)
Parameter containing:
tensor([0.5808], requires_grad=True)


In [10]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [11]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

In [12]:
loss = nn.MSELoss()

In [13]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [16]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000057
epoch 2, loss: 0.000066
epoch 3, loss: 0.000124


In [17]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

tensor([[ 2.0000],
        [-3.4000]]) Parameter containing:
tensor([[ 2.0008, -3.3999]], requires_grad=True)
tensor(4.2000) Parameter containing:
tensor([4.2004], requires_grad=True)
